In [229]:
from math import ceil
import time, re, ast, sys, urllib, random, string, requests
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from itertools import cycle
from header_list import user_agent_list
from proxies_list import proxies_list_
from INITIAL_SCRAPPER_FUNCTIONS import *
from PARSER_FUNCTIONS import *
from LIST_DF_FUNCTIONS import *
from SEARCH_URL_GEN import *
from GET_SEARCH_URL import *
from GET_RESULTS import *
ua = user_agent_list
proxies = proxies_list_

In [238]:
customer_url = input("Input the Redfin URL of the Home: ")

Input the Redfin URL of the Home: http://www.redfin.com/WA/Bainbridge-Island/12612-Kallgren-Rd-NE-98110/home/2198065


In [239]:
header = random.sample(ua, 1)[0]
proxy = random.sample(proxies, 1)[0]
url = customer_url
print(proxy)
print(url)
soup = session_creator(proxy, ua, url)

172.83.138.45:53281
http://www.redfin.com/WA/Bainbridge-Island/12612-Kallgren-Rd-NE-98110/home/2198065


In [240]:
customer_df = info_from_property(soup)

/home/guess/apt-get-home/src/PARSER_FUNCTIONS.py:308: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  all_dict = df.to_dict()


In [241]:
customer_df

,address,city,state,zip_code,sq_ft,latitude,longitude,redfin_est,last_sold_price,num_bdrs,...,zoning_jurisdiction,sq_ft_finished,preliminary_title_ordered,tax_id_#,extra_feats,prop_type,view_s,offer_review_date,community,mls#
1,12612 Kallgren Rd NE,"Bainbridge Island,",WA,98110,"2,960",47.6776916,-122.5152098,"$934,449","$941,800",4,...,City,"2,960",Yes,11250220812000,"Hardwood Floor, Composition Roof, Deck, Jetted...",Single Family Residence,Territorial,"Tuesday, June 12, 2018",Rolling Bay,1302171


In [242]:
c_url = gen_zip_url(customer_df)
search_url_part_eight_gen(1.02*43260)

'min-lot-size=0.25-acre,max-lot-size=3-acre'

In [243]:
customer_df['sq_ft']

1    2,960
Name: sq_ft, dtype: object

In [244]:
c_url

'https://www.redfin.com/zipcode/98110/filter/sort=lo-days,property-type=house,min-price=750k,max-price=1M,min-baths=1.25,min-sqft=2.5k-sqft,max-sqft=3.5k-sqft,min-year-built=1980,max-year-built=2010,min-lot-size=0.25-acre,max-lot-size=3-acre'

In [246]:
header = random.sample(ua, 1)[0]
proxy = random.sample(proxies, 1)[0]
print(proxy)
soup = session_creator(proxy, ua, c_url)
get_results(soup, customer_df)

205.169.145.130:39438
Found 9 properties
Here are top two homes out of 9:
Home # 1: https://www.redfin.com/WA/Bainbridge-Island/13690-Sunrise-Dr-NE-98110/home/2148184
Home # 2: https://www.redfin.com/WA/Bainbridge-Island/14235-Sunrise-Dr-NE-98110/home/2148490
